In [1]:
import logging
import uuid

from random import sample

from utils.ping import PING
from utils.atlas_api import get_measurements_from_tag
from utils.measurement_utils import load_pickle, dump_pickle, save_config_file, get_prefix_from_ip
from default import (
    ANCHORS_FILE,
    PROBES_FILE,
    HITLIST_FILE,
    PREFIX_PROBE_VP,
    TARGET_PROBE_VP,
    MEASUREMENT_CONFIG_PATH,
    RIPE_CREDENTIALS,
)

logging.basicConfig(level=logging.INFO)

NB_TARGETS = 2
NB_VPS = 4

#NBTARGETS = -1
#NB_VPS = -1

In [2]:
# load datasets

targets_per_prefix = load_pickle(HITLIST_FILE)

anchors = load_pickle(ANCHORS_FILE)

probes = load_pickle(PROBES_FILE)

## select targets and vps dataset

In [3]:
if NB_TARGETS > 0:
    targets = sample(list(anchors.keys()), k=NB_TARGETS)
else:
    targets = list(anchors.keys())

if NB_VPS > 0:
    vps = {}
    shuffle = sample(list(probes.keys()), k=NB_VPS)
    for probe in shuffle:
        vps[probe] = probes[probe]
else:
    vps = probes


logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 4


# Tier 1: probing each target prefixes

## measure target prefixes

In [4]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type": "prefix",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
save_config_file(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

try:
    cached_results = load_pickle(PREFIX_PROBE_VP)
    logging.info(
        f"initial length targets: {len(target_prefixes)}, cached measurements : {len(cached_results)}"
    )

    # get prefixes out of targets
    cached_results = [
        get_prefix_from_ip(target_addr) for target_addr in cached_results
    ]
    target_prefixes = list(
        set(target_prefixes).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(target_prefixes)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(target_prefixes)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_prefix(
    target_prefixes=target_prefixes,
    vps=vps,
    targets_per_prefix=targets_per_prefix,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
save_config_file(measurement_config, config_file)

INFO:root:No cached results available
INFO:root:Starting measurements d2b7f142-db88-45c2-be5c-c79c1098f59d with parameters: dry_run=False; nb_targets=2; nb_vps=4.
INFO:root:measurement : d2b7f142-db88-45c2-be5c-c79c1098f59d done


## retrieve prefix probing results 

In [5]:
measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))

# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_pickle(measurement_results, PREFIX_PROBE_VP)

nb measurements retrieved: 6
212.101.201.3 : 4
212.101.201.6 : 4
212.101.201.5 : 4
212.10.11.0 : 4
212.10.11.4 : 4
212.10.11.2 : 4


# Tier 2: probing each target

## measure targets

In [6]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type": "target",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
save_config_file(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)


try:
    cached_results = load_pickle(TARGET_PROBE_VP)
    logging.info(
        f"initial length targets: {len(targets)}, cached measurements : {len(cached_results)}"
    )

    targets = list(
        set(targets).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(targets)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(targets)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_target(
    targets=targets,
    vps=vps,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
save_config_file(measurement_config, config_file)

INFO:root:No cached results available
INFO:root:Starting measurements 3c9a548e-6263-405c-a658-44834f629845 with parameters: dry_run=False; nb_targets=2; nb_vps=4.
INFO:root:measurement : 3c9a548e-6263-405c-a658-44834f629845 done


## retrieve traget probing results

In [7]:
measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))

# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_pickle(measurement_results, TARGET_PROBE_VP)

nb measurements retrieved: 2
212.101.201.29 : 4
212.10.11.213 : 4
